In [ ]:
import keras
keras.__version__

'2.7.0'

# 2.3. The gears of neural networks: tensor operations

- All transformations learned by deep neural networks can be reduced to   
a handful of **tensor operations** applied to tensors of numeric data.  
(심층 신경망이 학습한 모든 변환을 **수치 데이터 텐서에 적용하는 몇 종류의 텐서 연산**으로 나타낼 수 있음)
- For instance, it’s possible to **add tensors, multiply tensors**, and so on.


In our initial example, we were building our network by **stacking Dense layers** on top of each other. 

A Keras layer instance looks like this:

In [ ]:
keras.layers.Dense(512, activation='relu')

- **This layer** can be interpreted as a **function**,  
which takes as **input a 2D tensor** and **returns another 2D tensor**  
 —**<font color="blue">a new representation for the input tensor**</font>.  
(이 층은 **2D 텐서를 입력**으로 받고  
 **입력 텐서의 새로운 표현**인 **또 다른 2D 텐서를 반환**하는 **함수**처럼 해석할 수 있음)  

         y=f(x)  
         New representations = dense(input tensor)



- Specifically, the function is as follows (where W is a 2D tensor and b is a vector, both attributes (parameters) of the layer):  
(함수는 다음과 같음 (**W는 2D 텐서, b는 벡터, 둘 모두 layer(층)의 속성**임))

**output = relu(dot(W, input) + b)**

Let’s unpack this. 

**We have three tensor operations here:** 
- a **dot product (dot)** between the input tensor and a tensor named W; 
- an **addition (+)** between the resulting 2D tensor and a vector b; 
- and, finally, **a relu operation**. relu(x) is max(x, 0).

## 2.3.1. Element-wise operations

- The **relu operation** and **addition** are **element-wise operations**: 
- That is, operations that are applied independently to each entry in the tensors being considered.  
(이 연산은 텐서에 있는 **각 원소에 독립적으로 적용**됨)
- This means these operations are **highly amenable to massively parallel implementations**  
(vectorized implementations, a term that comes from the vector processor supercomputer architecture from the 1970–1990 period).  
(이 말은 **고도의 병렬 구현이 가능**한 연산이라는 의미임.) 


<div>
<img src="https://drive.google.com/uc?export=view&id=1kYZcfREcHsqOtjItmicEXcwByG6UkGhm" width="500"/>
</div>
[그림출처:https://www.sharpsightlabs.com/blog/numpy-relu/]

<div>
<img src="https://drive.google.com/uc?export=view&id=1brqTic1gberwaEo15QG2PylvVOLktZcw" width="400"/>
</div>
[그림출처:https://pyimagesearch.com/2021/05/14/convolutional-neural-networks-cnns-and-layer-types/]

<div>
<img src="https://drive.google.com/uc?export=view&id=1c8R6EKdEWa2hVbEYlBIWAlbW6jhXuaKl" width="600"/>
</div>
[그림출처:https://www.cuemath.com/algebra/addition-of-matrices/]


If you want to write a naive Python implementation of an element-wise operation, you use a for loop, 

as in this **naive implementation of an element-wise relu operation**:

In [ ]:
def naive_relu(x):
    assert len(x.shape) == 2                     #  x is a 2D Numpy tensor.

    x = x.copy()                                 #  Avoid overwriting the input tensor.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

You do the same for **addition**:

In [ ]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape                # x and y are 2D Numpy tensors.

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

On the same principle, you can do **element-wise multiplication**, **subtraction**, and so on.

- In practice, when dealing with Numpy arrays, these operations are available as well-optimized **built-in Numpy functions**

- So, in Numpy, you can do the following element-wise operation, and it will be blazing fast:

In [ ]:
import numpy as np

z = x + y                       # Element-wise Addition

z = np.maximum(z, 0.)           # Element-wise relu

## 2.3.2. Broadcasting

- Our earlier naive implementation of naive_add only supports 
  the addition of **2D tensors with identical shapes**.  
  (앞서 살펴본 단순한 덧셈 구현인 naive_add는 **동일한 크기**의 2D 텐서만 지원함)
- But in the **Dense layer** introduced earlier, <font color="blue">**we added a 2D tensor with a vector**</font>.  
(하지만 이전에 보았던 **Dense 층에서는 2D 텐서와 벡터를 더했음**)

### **What happens with addition when the shapes of the two tensors being added differ?**  
**크기가 다른 두 텐서가 더해질 때 무슨 일이 일어날까요?**

When possible, and if there’s no ambiguity, 

- the smaller tensor will be broadcasted to match the shape of the larger tensor.  
(<font color="blue">**작은 텐서가 큰 텐서의 크기에 맞추어 브로드캐스팅(broadcasting)됨**</font>)

### Broadcasting consists of two steps:

1. Axes (called broadcast axes) are added to the smaller tensor to match the ndim of the larger tensor.  
(**큰 텐서의 ndim에 맞도록 작은 텐서에 축이 추가**됨. 이 추가된 새 축을 **브로드캐스팅 축**이라고 함)
1. The smaller tensor is repeated alongside these new axes to match the full shape of the larger tensor.  
(**작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됨**) 

<div>
<img src="https://drive.google.com/uc?export=view&id=1SDbse23OO0gdJ2kK-4tTGa2WhPkdEmyd" width="500"/>
</div>
[그림출처:https://medium.com/analytics-vidhya/pytorch-for-deep-learning-part-2-bc0cfa12e74]


### Let’s look at a concrete example. 

- Consider X with shape (32, 10) and y with shape (10,). 
- First, we add an empty first axis to y, whose shape becomes (1, 10). 
- Then, we repeat y 32 times alongside this new axis, so that we end up with a tensor Y with shape (32, 10), where Y[i, :] == y for i in range(0, 32). 
- At this point, we can proceed to add X and Y, because they have the same shape.


### Here’s what a naive implementation would look like:

In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2                           #  x is a 2D Numpy tensor
    assert len(y.shape) == 1                           #  y is a Numpy vector.
    assert x.shape[1] == y.shape[0]

    x = x.copy()                                       #  Avoid overwriting the input tensor
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

- With broadcasting, you can generally apply two-tensor element-wise operations if one tensor has shape  
(**a, b, ... , n-1**, n, n + 1, ... , m) and the other has shape (n, n + 1, ..., m). 

- The broadcasting will then automatically happen for axes **a through n - 1**.

### The following example applies the element-wise maximum operation to two tensors of different shapes via broadcasting:

In [ ]:
import numpy as np

x = np.random.random((64, 3, 32, 10))        # x is a random tensor with shape (64, 3, 32, 10)
y = np.random.random((32, 10))              #  y is a random tensor with shape (32, 10).

z = np.maximum(x, y)                        # The output z has shape (64, 3, 32, 10) like x

In [ ]:
z.shape

(64, 3, 32, 10)

## 2.3.3. Tensor dot (텐서 점곱)

- The **dot operation**, also called a **tensor product** (not to be confused with an element-wise product) is the most common, most useful tensor operation.  
(텐서 곱셈(tensor product)라고도 부르는 점곱 연산(dot operation)은 가장 널리 사용되고 유용한 텐서 연산임)
- Contrary to element-wise operations, <font color="blue">**it combines entries in the input tensors**</font>.  
(원소별 연산과 반대로 입력 텐서의 원소들을 겹합시킴)

- An element-wise product is done with the * operator in Numpy, Keras, Theano, and TensorFlow. 
- dot uses a different syntax in TensorFlow, but in both Numpy and Keras it’s done using the standard dot operator:

@ 텐서플로에서는 tf.matmul(x, y)처럼 사용함.

In [ ]:
import numpy as np
z = np.dot(x, y)

In [ ]:
from keras import backend as K
z = K.dot(x, y)

:In mathematical notation, you’d note the operation with a dot (.)

z = x . y

Mathematically, what does the dot operation do? 
Let’s start with the dot product of two vectors x and y. 

It’s computed as follows:

In [ ]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]

    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z
     
    # x and y are Numpy vectors.

- The dot product between two vectors is a **scalar**.  
(두 벡터의 dot product는 스칼라가 됨)
- Only vectors with **the same number** of elements are compatible for a dot product.  
(원소 개수가 같은 벡터끼리 dot product가 가능함)

You can also take **the dot product between a matrix x and a vector y**, which returns **a vector where the coefficients are the dot products between y and the rows of x**.  
(행렬 x와 벡터 y 사이에서도 dot product가 가능함  
 y와 x의 행 사이에서 dot product가 일어나므로 벡터가 반환됨)


You implement it as follows:

In [ ]:
import numpy as np

def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]

    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

You could also reuse the code we wrote previously, which highlights the relationship between a matrix-vector product and a vector product:

In [ ]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

Note that as soon as one of the two tensors has an ndim greater than 1, 

dot is no longer symmetric (commutative), which is to say that <font color="blue">**dot(x, y) isn’t the same as dot(y, x)**</font>.  
(두 텐서 중 하나라도 ndim이 1보다 크면 dot 연산에 교환 법칙이 성립되지 않음. 다시 말하면, dot(x,y)와 dot(y,x)가 일반적으로 같지 않음.)

- Of course, a dot product generalizes to tensors with an arbitrary number of axes.  
(dot product는 임의의 축 개수를 가진 텐서에 일반화됨)
- The most common applications may be the dot product between two matrices.  
(가장 일반적인 용도는 두 행렬 간의 dot product임)
- You can take the dot product of two matrices x and y (dot(x, y)) if and only if x.shape[1] == y.shape[0].  
(x.shape[1] == y.shape[0]일 때, 두 행렬 x와 y의 dot product (dot(x, y))가 성립함)
- The result is a matrix with shape (x.shape[0], y.shape[1]), where the coefficients are the vector products between the rows of x and the columns of y.  
(x의 열과 y의 행 사이 벡터 dot product로 인해 (x.shape[0], y.shape[1]) 크기의 행렬이 됨)

Here’s the naive implementation:

In [ ]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]

    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

To understand dot-product shape compatibility, it helps to visualize the input and output tensors by aligning them as shown in figure 2.5.


<div>
<img src="https://drive.google.com/uc?export=view&id=1GQOqhqNV42cZUPuwLcgd0BgXH5gcu516" width="600"/>
</div>

- x, y, and z are pictured as rectangles (literal boxes of coefficients). 
- Because the rows x and the columns of y must have the same size, it follows that the width of x must match the height of y.

More generally, you can take the dot product between higher-dimensional tensors, following the same rules for shape compatibility as outlined earlier for the 2D case  
(더 일반적으로는 앞서 설명한 2D의 경우처럼 크기를 맞추는 동일한 규칙을 따르면 다음과 같이 고차원 텐서 간의 dot product을 할 수 있음):

<font color="blue">**(a, b, c, d) . (d,) -> (a, b, c)**</font>

<font color="blue">**(a, b, c, d) . (d, e) -> (a, b, c, e)**</font>

## 2.3.4. Tensor reshaping

-  It is essential to understand is **tensor reshaping (텐서 크기 변환)**. 
- Although it wasn’t used in the Dense layers in our first neural network example, we used it when we preprocessed the digits data before feeding it into our network:  
(2.1 예제에서 신경망에 주입할 숫자 데이터를 전처리할 때 사용함)

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images.shape

(60000, 28, 28)

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))

In [ ]:
train_images.shape

(60000, 784)

- **Reshaping a tensor means rearranging its rows and columns to match a target shape**.  
(<font color="blue">텐서의 크기를 변환한다는 것은 특정 크기에 맞게 열과 행을 재배열한다는 뜻임</font>) 
- Naturally, the reshaped tensor has the same total number of coefficients as the initial tensor.  
(<font color="blue">변환된 텐서는 원래 텐서와 원소의 개수가 동일함.</font>)

Reshaping is best understood via simple examples:

In [ ]:
import numpy as np
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print(x.shape)

(3, 2)


In [ ]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [ ]:
x.shape

(6, 1)

In [ ]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

In [ ]:
x.shape

(2, 3)

- A special case of reshaping that’s commonly encountered is **transposition**.  
(자주 사용하는 특별한 크기 변환은 전치(transposition)임)
- Transposing a matrix means **exchanging its rows and its columns**, so that x[i, :] becomes x[:, i]:  
(행렬의 전치는 행과 열을 바꾸는 것을 의미함. 즉, x[i, :]이 x[:, i]가 됨)

In [ ]:
x = np.zeros((300, 20))

x = np.transpose(x)

print(x.shape)

(20, 300)


## 2.3.5. Geometric interpretation of tensor operations

- Because the contents of the tensors manipulated by tensor operations can be interpreted as coordinates of points in some geometric space, **all tensor operations have a geometric interpretation**.  
(<font color="blue">텐서 연산이 조작하는 **텐서의 내용**은 **어떤 기하학적 공간에 있는 좌표 포인트**로 해석될 수 있기 때문에  
 **모든 텐서 연산은 기하학적 해석이 가능**함.</font>) 
- For instance, let’s consider addition. We’ll start with the following vector:

In [ ]:
A = [0.5, 1]

It’s a point in a 2D space (see figure 2.6). It’s common to picture a vector as an arrow linking the origin to the point, as shown in figure 2.7.

<div>
<img src="https://drive.google.com/uc?export=view&id=1osN0uZTXZRBplCAUQF-hrSoov6KPwbEu" width="600"/>
</div>

<div>
<img src="https://drive.google.com/uc?export=view&id=1ivvabBIiG61w4Xg4A-RhmsHpwzua2WSL" width="600"/>
</div>

- Let’s consider a new point, B = [1, 0.25], which we’ll add to the previous one. 
- This is done geometrically by chaining together the vector arrows, with the resulting location being the vector representing the sum of the previous two vectors (see figure 2.8).  
(기하학적으로 벡터 화살표를 연결하여 계산할 수 있음. 최종 위치는 두 벡터의 덧셈을 나타내는 벡터가 됨)

<div>
<img src="https://drive.google.com/uc?export=view&id=1JYlST6gCaSI2YOhf8po_eDuXBhZVk9UC" width="600"/>
</div>

- In general, **elementary geometric operations** such as affine transformations, rotations, scaling, and so on can be expressed as **tensor operations.**  
(일반적으로 아핀 변환(affine transformation), 회전, 스케일링 등 처럼 <font color="blue">**기본적인 기하학적 연산은 텐서 연산으로 표현될 수 있음**</font>)
- For instance, a rotation of a 2D vector by an angle theta can be achieved via a dot product with a 2 × 2 matrix R = [u, v], where u and v are both vectors of the plane: u = [cos(theta), sin(theta)] and v = [-sin(theta), cos(theta)].  
(예를 들어, theta 각도로 2D 벡터를 회전하는 것은 2x2 행렬 R = [u, v]를 dot product하여 구현할 수 있음. 여기에서 u 와 v 는 동일 평면상의 벡터이며, u = [cos(theta), sin(theta)] 고 v = [-sin(theta), cos(theta)]임)

## 2.3.6. A geometric interpretation of deep learning

You just learned that 

- **Neural networks consist entirely of chains of tensor operations**.  
(신경망은 전체적으로 텐서 연산의 연결로 구성됨)
- All of these tensor operations are just **geometric transformations of the input data**.  
(모든 텐서 연산은 입력 데이터의 기하학적 변환임)
- It follows that you can interpret a neural network as **a very complex geometric transformation in a high-dimensional space**, implemented via a long series of simple steps.  
(단순한 단계들이 길게 이어져 구현된 신경망을 <font color="blue">**고차원 공간에서 매우 복잡한 기하학적 변환**</font>을 하는 것으로 해석 가능)

In 3D, the following mental image may prove useful. 

- Imagine two sheets of colored paper: one red and one blue. 
- Put one on top of the other. 
- Now crumple them together into a small ball. 
- That crumpled paper ball is your input data, and each sheet of paper is a class of data in a classification problem. 
- What a neural network (or any other machine-learning model) is meant to do is figure out a transformation of the paper ball that would uncrumple it, so as to make the two classes cleanly separable again. 
- With deep learning, this would be implemented as a series of simple transformations of the 3D space, such as those you could apply on the paper ball with your fingers, one movement at a time.

Figure 2.9. Uncrumpling a complicated manifold of data
<div>
<img src="https://drive.google.com/uc?export=view&id=12gC5q90kT5vOo_N0l0rujv65xqExIvGg" width="800"/>
</div>

- Uncrumpling paper balls is what machine learning is about: **finding neat representations for complex, highly folded data manifolds**. (즉, 머신러닝 : 복잡하고 심하게 꼬여 있는 데이터의 manifold에 대한 깔끔한 표현을 찾는 일을 함.)
- At this point, you should have a pretty good intuition as to why deep learning excels at this: it takes the approach of incrementally decomposing a complicated geometric transformation into a long chain of elementary ones, which is pretty much the strategy a human would follow to uncrumple a paper ball.(딥러닝이 뛰어난 이유는 기초적인 연산을 길게 연결하여 복잡한 기하학적 변환을 조금씩 분해하는 방식이 마치 사람이 종이 공을 펼치기 위한 전략과 매우 흡사하기 때문임.) 
- Each layer in a deep network applies a transformation that disentangles the data a little—and a deep stack of layers makes tractable an extremely complicated disentanglement process.(심층 네트워크의 각 층은 데이터를 조금씩 풀어 주는 변환을 적용하므로, 이런 층을 깊게 쌓으면 아주 복잡한 분해 과정을 처리할 수 있다.)